In [ ]:
import json
import pickle
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import google.generativeai as genai


## Embedding and Indexing

In [4]:
# Hàm trích xuất văn bản từ sub_content
def extract_text_from_subcontent(subcontent):
    """
    subcontent có thể là:
    - Một danh sách các chuỗi
    - Một danh sách các dict (có thể chứa subheader, paragraph, items, details, v.v.)
    Hàm này trả về danh sách các đoạn văn bản.
    """
    results = []

    # Trường hợp sub_content toàn chuỗi
    if all(isinstance(item, str) for item in subcontent):
        results.extend(subcontent)
    else:
        # Trường hợp sub_content chứa các dict
        for elem in subcontent:
            if isinstance(elem, dict):
                # Xử lý subheader
                if "subheader" in elem and elem["subheader"]:
                    results.append(elem["subheader"])
                # Xử lý paragraph
                if "paragraph" in elem and elem["paragraph"]:
                    results.append(elem["paragraph"])
                # Xử lý items
                if "items" in elem and isinstance(elem["items"], list):
                    for item in elem["items"]:
                        if "paragraph" in item and item["paragraph"]:
                            results.append(item["paragraph"])
                        if "details" in item and item["details"]:
                            results.extend(item["details"])
                # Xử lý details
                if "details" in elem and elem["details"]:
                    results.extend(elem["details"])

    return results


In [5]:
# Hàm tạo embeddings và lưu vào file pickle
def create_indexed_data(json_file, model, output_pickle):
    """
    Đọc file JSON, tạo embeddings cho từng chunk (header + nội dung) và lưu kết quả vào file pickle.
    """
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    chunks = []
    for idx, item in enumerate(data):
        # Trích xuất header và nội dung
        header_name = item["header"]
        extracted_text = extract_text_from_subcontent(item["sub_content"])
        combined_text = f"{header_name}\n" + "\n".join(extracted_text)
        chunks.append({
            "header_index": idx,
            "header": header_name,
            "text": combined_text
        })

    # Tạo embeddings cho các đoạn văn bản
    texts = [chunk["text"] for chunk in chunks]
    embeddings = model.encode(texts, show_progress_bar=True)

    # Lưu kết quả
    indexed_data = []
    for i, emb in enumerate(embeddings):
        indexed_data.append({
            "header_index": chunks[i]["header_index"],
            "header": chunks[i]["header"],
            "text": chunks[i]["text"],
            "embedding": emb
        })

    # Lưu indexed data ra file pickle
    with open(output_pickle, "wb") as f:
        pickle.dump(indexed_data, f)

    print(f"Indexed {len(indexed_data)} chunks.")


In [ ]:
if __name__ == "__main__":
    # Load mô hình SentenceTransformer
    model = SentenceTransformer("all-MiniLM-L6-v2")
    
    # File JSON input và file pickle output
    json_file = "cleaned_data.json"
    output_pickle = "indexed_data.pkl"

    # Tạo indexed data
    create_indexed_data(json_file, model, output_pickle)


## Chatbot

In [6]:
# Hàm tìm các chunk phù hợp nhất với câu hỏi người dùng
def find_best_chunks(question, model, indexed_data, header_weight=3.0, content_weight=2.0, k=2):
    """
    Tìm các đoạn văn phù hợp nhất bằng cách ưu tiên header, sau đó fallback về nội dung.
    - Sử dụng trọng số cho header và nội dung.
    - Trả về top-k đoạn phù hợp.
    """
    # Tạo embedding cho câu hỏi
    question_embedding = model.encode([question])

    # Tính toán độ tương đồng cho header và nội dung
    header_embeddings = np.vstack([item["embedding"] for item in indexed_data])
    content_embeddings = np.vstack([model.encode([item["text"]])[0] for item in indexed_data])

    header_similarities = cosine_similarity(question_embedding, header_embeddings)[0]
    content_similarities = cosine_similarity(question_embedding, content_embeddings)[0]

    # Tính điểm tổng hợp với trọng số
    combined_scores = header_weight * header_similarities + content_weight * content_similarities

    # Lấy top-k đoạn văn bản
    sorted_indices = np.argsort(combined_scores)[::-1]
    top_indices = sorted_indices[:k]

    return [indexed_data[i] for i in top_indices]


In [7]:
# Hàm gửi nội dung tới Gemini và lấy câu trả lời
def generate_answer(question, chunks):
    """
    Kết hợp các đoạn văn bản và gửi tới Gemini để sinh câu trả lời.
    """
    context = "\n\n---\n\n".join([chunk["text"] for chunk in chunks])
    prompt = f"""
You are an AI assistant. Use the provided context to answer the user's question in English.

Context:
{context}

User's Question:
{question}

Answer in natural language:
"""
    # Gửi prompt tới Gemini
    chat_session = genai.GenerativeModel(model_name="gemini-2.0-flash-exp").start_chat()
    response = chat_session.send_message(prompt)
    return response.text.strip()


In [8]:
# Hàm giao diện chatbot CLI
def chatbot(model, indexed_data):
    print("Hello! Ask me anything about the privacy policy. Type 'exit' to quit.")
    while True:
        user_question = input("\nYour question: ")
        if user_question.lower() == "exit":
            print("Goodbye!")
            break

        # Tìm các đoạn phù hợp nhất
        best_chunks = find_best_chunks(user_question, model, indexed_data, header_weight=3.0, content_weight=2.0, k=3)
        print(f"Found relevant chunks:\n{[chunk['header'] for chunk in best_chunks]}")

        # Gửi câu hỏi và nhận câu trả lời từ Gemini
        answer = generate_answer(user_question, best_chunks)
        print("\nAnswer:", answer)


In [9]:
if __name__ == "__main__":

    GENAI_API_KEY = os.getenv("GENAI_API_KEY")
    if GENAI_API_KEY:
        print("API Key successfully retrieved!")
    else:
        print("API Key not found. Please check your environment variables.")
    print(f"Your API Key: {GENAI_API_KEY}")

    genai.configure(api_key=GENAI_API_KEY)

    # Load mô hình và indexed data
    model = SentenceTransformer("all-MiniLM-L6-v2")
    with open("indexed_data.pkl", "rb") as f:
        indexed_data = pickle.load(f)

    # Chạy chatbot
    chatbot(model, indexed_data)

c:\Users\Phuong\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Hello! Ask me anything about the privacy policy. Type 'exit' to quit.
Goodbye!
